In [1]:
import SimpleITK as sitk
import nibabel as nib
import os
import time
import numpy as np

In [5]:
r = 10
bs = 128
data_dir = "/notebooks/neuromod/derivatives/deepneuroan/training/generated_data"
template_filepath = os.path.join(data_dir, "template_on_grid")

list_files = []
list_files_tmp = set([])
for root, _, files in os.walk(data_dir):
    for file in files:
        filepath = os.path.join(root, file).split('.')[0]
        if os.path.exists(filepath + ".txt"):
            list_files_tmp.add(filepath)
list_files = list(list_files_tmp)
list_files = list_files[:r]

def normalize_img(img):
    return (img - np.mean(img)) / np.std(img)

big_img = np.zeros((bs, 220, 220, 220))

# simple itk
tic = time.time()
for idx in range(r):
    print(idx)
    for i, file in enumerate(list_files):
        img = sitk.GetArrayFromImage(sitk.ReadImage(file + ".nii.gz", sitk.sitkFloat32))
        big_img[i,] = normalize_img(img)
ElpsTime = time.time() - tic
print("*** Total sitk %1.2f s ***"%(ElpsTime))
print("%1.4f s per sample"%(ElpsTime/(r*bs)))

# nibabel
tic = time.time()
for idx in range(r):
    print(idx)
    for i, file in enumerate(list_files):
        img = nib.load(file + ".nii.gz").get_data()
        big_img[i,] = normalize_img(img)
ElpsTime = time.time() - tic
print("*** Total nibabel %1.2f s ***"%(ElpsTime))
print("%1.4f s per sample"%(ElpsTime/(r*bs)))

0
1
2
3
4
5
6
7
8
9
*** Total sitk 15.48 s ***
0.0121 s per sample
0
/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-192_transfo-000008.nii
/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-116_transfo-000007.nii
/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-104_transfo-000007.nii
/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-106_transfo-000001.nii
/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-109_transfo-000010.nii
/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-088_transfo-000007.nii
/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-021_transfo-000006.nii
/notebooks/neuromod/derivati

/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-109_transfo-000010.nii
/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-088_transfo-000007.nii
/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-021_transfo-000006.nii
/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-112_transfo-000007.nii
/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-190_transfo-000006.nii
/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-137_transfo-000002.nii
7
/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-192_transfo-000008.nii
/notebooks/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-0